In [ ]:
import fitz  # PyMuPDF
import re
import csv

pdf_file_path = r'C:\Users\User\Dropbox\textmining\PDF\Files (35).PDF'
csv_file_path = r'C:\Users\User\Dropbox\textmining\opinion\opinion_35_Test.csv'

# 更新後的正則表達式，確保遇到下一個標記或"Opinion"時停止，並排除Footnotes的內容
pattern = r'(Counsel:.*?)(?=Counsel:|Judges:|Opinion by:|Opinion|$)|(Judges:.*?)(?=Counsel:|Judges:|Opinion by:|Opinion|$)|(Opinion by:.*?)(?=Counsel:|Judges:|Opinion by:|Opinion|$)'

def find_page_num(text, match_start, page_delimiters):
    for i, delimiter in enumerate(page_delimiters):
        if match_start < delimiter:
            return i
    return len(page_delimiters)

counsel_count = 0
judges_count = 0
opinion_by_count = 0

with fitz.open(pdf_file_path) as doc:
    all_text = ""
    page_delimiters = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        page_text = page.get_text("text")
        all_text += page_text + "\f"
        page_delimiters.append(len(all_text))

    # 移除頁碼和案例引用
    all_text = re.sub(r'Page \d+ of \d+', '', all_text)
    all_text = re.sub(r'\d+ Cal\..*?\*{1,4}\d+; (?:\d{4} )?Cal\..*?\*{1,4}\d+', '', all_text)

    with open(csv_file_path, 'w', encoding='utf-8', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['頁碼', '內容類型', '內容'])

        matches = re.finditer(pattern, all_text, re.DOTALL)
        for match in matches:
            for group_num in range(1, 4):
                if match.group(group_num):
                    match_content = match.group(group_num).strip()
                    match_page = find_page_num(all_text, match.start(group_num), page_delimiters)
                    content_type = 'Counsel' if group_num == 1 else 'Judges' if group_num == 2 else 'Opinion by'
                    csv_writer.writerow([match_page + 1, content_type, match_content])
                    if content_type == 'Counsel':
                        counsel_count += 1
                    elif content_type == 'Judges':
                        judges_count += 1
                    elif content_type == 'Opinion by':
                        opinion_by_count += 1
                    break

print(f"完成：{csv_file_path}")
print(f"Counsel 數量: {counsel_count}")
print(f"Judges 數量: {judges_count}")
print(f"Opinion by 數量: {opinion_by_count}")
